In [9]:
!pip install pymupdf

In [32]:
import fitz  
import pandas as pd
import re

def extract_text_from_pdf(pdf_path):
    # Open the PDF file
    pdf_document = fitz.open(pdf_path)
    articles = []

    article_number_pattern = re.compile(r'^\d+\.\d+\.')  # Pattern to match article numbers like 1.1, 1.2, etc.
    current_article = {"s_no": "", "article_title": "", "article_body": ""}
    collecting_body = False

    for page_num in range(6,len(pdf_document)):
        page = pdf_document[page_num]
        text = page.get_text("text").encode("utf-8-sig").decode("utf-8-sig")
        lines = text.splitlines()

        for line in lines:
            line = line.strip()
            if article_number_pattern.match(line):
                if collecting_body and current_article["s_no"]:  # Save the previous article before starting a new one
                    articles.append(current_article)

                # Extract article number and title
                parts = line.split('.', 2)
                current_article = {"s_no": f"{parts[0]}.{parts[1]}.", "article_title": parts[2].strip('" '), "article_body": ""}
                collecting_body = True
            elif collecting_body:
                if current_article["article_body"]:
                    current_article["article_body"] += "\n" + line
                else:
                    current_article["article_body"] = line

    if current_article["s_no"]:  # Save the last article
        articles.append(current_article)

    return articles

def save_to_csv(articles, csv_path):
    for article in articles:
        article["article_title"] = f'"{article["article_title"]}"'
        article["article_body"] = f'"{article["article_body"]}"'
    df = pd.DataFrame(articles)
    df.to_csv(csv_path, index=False)

if __name__ == "__main__":
    pdf_path = r"C:\Users\Rohith\Downloads\VisionIAS Monthly Current Affairs January 2024 January 2024.pdf"  # Update this to the path of your PDF file
    csv_path = r"C:\Users\Rohith\Downloads\articles.csv"  # Update this to the desired CSV file path

    articles = extract_text_from_pdf(pdf_path)
    save_to_csv(articles, csv_path)

    print(f"Extracted {len(articles)} articles and saved to {csv_path}")

Extracted 167 articles and saved to C:\Users\Rohith\Downloads\articles.csv
